<a href="https://colab.research.google.com/github/vaibhav-108/GENAI_basic/blob/main/RAG_with_History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00


In [19]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('OPENAI_API')
LANGCHAIN_API_KEY=userdata.get('langchain')

import os
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
os.environ["LANGCHAIN_API_KEY"]=LANGCHAIN_API_KEY

In [4]:
!pip install -U langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI


from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model="google/gemini-2.5-pro-exp-03-25",
                 api_key=GOOGLE_API_KEY,
                 openai_api_base="https://openrouter.ai/api/v1")



<ipython-input-5-1b8e1d15e953>:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="google/gemini-2.5-pro-exp-03-25",


In [6]:
llm.invoke("what is a value of 2+3?").content

'2 + 3 = **5**.'

In [7]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4

In [8]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs=loader.load()
len(docs)

1

In [9]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits =text_splitter.split_documents(docs)

In [11]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}

embedding_function=HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs,
)

<ipython-input-11-88d32549705d>:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function=HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
vectorstore = Chroma.from_documents(splits,embedding_function)
retriever=vectorstore.as_retriever()

In [77]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [78]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7ea7cb477150>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7ea7cb0c5210>, model_name='google/gemini-2.5-pro-exp-03-25', model_kwargs={}, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')

In [79]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

In [80]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [116]:
rag_chain=(
    {"context" : retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [117]:
rag_chain.invoke("can you give me a Types of Memory?")

'Based on the provided text, the main types of human memory are Sensory Memory, Short-Term Memory (also called Working Memory), and Long-Term Memory. Long-Term Memory is further categorized into Explicit (declarative) memory, which includes episodic and semantic memory, and Implicit (procedural) memory. These types differ in how information is acquired, stored, retained, and retrieved.'

In [118]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [119]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

In [120]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [121]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_q_prompt,
)

In [122]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [123]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""


In [124]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [125]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [126]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [159]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history=[]

In [161]:
question1= "What is Task Decomposition?"
response1=rag_chain.invoke({"input":question1,"chat_history": chat_history})

In [135]:
response1

{'input': 'What is Task Decomposition?',
 'chat_history': [],
 'context': [Document(id='59b493f4-c2ff-4e4a-96de-a79df172a9a4', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'),
  Document(id='2fd52014-cd3c-421f-8968-510f5fcffd7e', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, p

In [136]:
response1["answer"]

'Task decomposition is the process of breaking down a complex task into smaller, simpler, and more manageable steps. This approach helps AI agents plan and handle difficult tasks more effectively. Techniques like Chain of Thought, which prompts the model to "think step by step," are used to achieve task decomposition.'

In [162]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=response1["answer"])
    ]
)

In [163]:
chat_history

[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Task decomposition is the process of breaking down a complicated task into smaller, simpler steps. This approach helps make large tasks more manageable for an AI agent. It is often achieved using prompting techniques like Chain of Thought, which instructs the model to "think step by step".', additional_kwargs={}, response_metadata={})]

In [166]:
question2="what are common ways of doing it?"
response2=rag_chain.invoke({"input":question2,"chat_history": chat_history})


In [167]:
response2

{'input': 'what are common ways of doing it?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Task decomposition is the process of breaking down a complicated task into smaller, simpler steps. This approach helps make large tasks more manageable for an AI agent. It is often achieved using prompting techniques like Chain of Thought, which instructs the model to "think step by step".', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='59b493f4-c2ff-4e4a-96de-a79df172a9a4', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with 

In [145]:
response2["answer"]

'Common ways to perform task decomposition include using a Large Language Model (LLM) with simple prompts like "Steps for XYZ?" or using specific instructions tailored to the task, such as "Write a story outline." Another method involves using human input to define the sub-tasks. Additionally, Chain of Thought prompting instructs the model to "think step by step" to break down the task.'

In [168]:
def chatbot(question):
  chat_hist=[]
  response=rag_chain.invoke({"input":question,"chat_history": chat_hist})
  chat_hist.extend(
    [HumanMessage(content=question1),
     AIMessage(content=response1["answer"])
    ])

  return response["answer"]


In [147]:
q1= "What is Task Decomposition?"
chatbot(q1)

'Task decomposition is the process of breaking down a complex task into smaller, more manageable steps or subgoals. This technique helps AI agents plan and tackle difficult problems more effectively. It can be achieved through methods like Chain of Thought prompting, using specific instructions, or via human input.'

In [148]:
q= "What are common ways of doing it?"
chatbot(q)

'Common ways to perform task decomposition include using a Large Language Model (LLM) with simple prompts asking for steps or subgoals. Another method is to employ task-specific instructions tailored to the problem, like asking for a story outline. Lastly, task decomposition can be achieved through direct human input.'

In [169]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [170]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition involves breaking down complex tasks into smaller, simpler, and more manageable steps. This process helps agents, like AI models, to plan and handle intricate problems more effectively. It can be achieved through methods like Chain of Thought prompting, using specific instructions, or via human input.'

In [173]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

TypeError: 'NoneType' object is not iterable

In [156]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 1. Modify your original rag_chain to use proper input keys
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),  # Receives the "input" directly
        "chat_history": RunnablePassthrough()["chat_history"]
    }
    | ChatPromptTemplate.from_template(
        """Answer the question based on the context and chat history:

        Chat History: {chat_history}

        Context: {context}

        Question: {question}"""
    )
    | llm
    | StrOutputParser()
)

# 2. Create the conversational chain with proper configuration
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",  # Matches the chain's expected input
    history_messages_key="chat_history",
)

# 3. Invoke with proper input structure
response = conversational_rag_chain.invoke(
    {"question": "What is Task Decomposition?"},
    config={"configurable": {"session_id": "abc123"}}
)

TypeError: 'RunnablePassthrough' object is not subscriptable